In [9]:
import requests 
import urllib.request
import pandas as pd
import numpy as np
import bs4
import json
import time

from itertools import count

MCD_URL = 'https://www.mcdonalds.co.kr/kor/store/main.do'

In [14]:
import requests
import urllib.request
import pandas as pd
import numpy as np
import bs4
import json
import time

from itertools import count

name_of_distr = ['경기도','서울특별시','인천광역시','경상남도','강원도','충청남도','대전광역시','충청북도','부산광역시','울산광역시','대구광역시','경상북도','전라남도','광주광역시','전라북도','제주특별자치도']

def get_mcd_address_one_page(location, page):
    response = urllib.request.urlopen(MCD_URL.format(location=urllib.parse.quote(location), page = page))
    mcd_data = response.read().decode('utf-8')
    soup = bs4.BeautifulSoup(mcd_data)
    ret = []
    for storetag in soup.findAll('dl', attrs={'class': 'clearFix'}):
        storename = storetag.findAll('a')[0].contents[-1].strip()
        storeaddr = storetag.findAll('dd', attrs={'class': 'road'})[0].contents[0].split(']')[1]
        storeaddr_district = storeaddr.split()[:2]
        ret.append([storename] + storeaddr_district)

    return pd.DataFrame(ret, columns=('store', 'd1', 'd2')) if ret else None

def search_mcdonalds_stores(location):
    found = []
        
    for pg in count():
        foundinpage = get_mcd_address_one_page(location, pg+1)
        if foundinpage is None:
            break
        found.append(foundinpage)
    
    return pd.concat(found)

found = []
for distr in name_of_distr:
    print("process",distr)
    found.append(search_mcdonalds_stores(distr))
mcd_tbl = pd.concat(found)
mcd_tbl['store'].value_counts().head()

process 경기도


ValueError: No objects to concatenate